In [1]:
import pandas as pd
from sklearn.utils import shuffle

In [2]:
data_rt = pd.read_csv("C:/Users/Valentina/Desktop/UDSC/Sentiment analysis/Data/reviews_rt_all.csv", sep="|")
data_imdb = pd.read_csv("C:/Users/Valentina/Desktop/UDSC/Sentiment analysis/Data/imdb_small.csv", sep="|")

data_df = pd.concat([data_rt, data_imdb], ignore_index=True, copy=False)
data_df = shuffle(data_df)


In [3]:
import re, nltk
from sklearn.feature_extraction.text import CountVectorizer        
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

In [6]:
from nltk.stem import SnowballStemmer
def tokenize(text):
    text = re.sub("[^a-zA-Z]", " ", text)
    word_list = nltk.word_tokenize(text)
    stemmer = SnowballStemmer("english")
    stems = [stemmer.stem(word) for word in word_list]
    return stems

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(tokenizer=tokenize, ngram_range=(1, 2),
                              analyzer = 'word', binary = True)

data_features = vectorizer.fit_transform(data_df['text'])

In [26]:
from sklearn.model_selection import train_test_split
X_train_rt, X_test_rt, y_train_rt, y_test_rt  = train_test_split(
        data_rt.text, 
        data_rt.label,
        test_size=0.2, 
        random_state=42)

In [27]:
X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb  = train_test_split(
        data_imdb.text, 
        data_imdb.label,
        test_size=0.2, 
        random_state=42)

In [28]:
X_train = pd.concat([X_train_rt, X_train_imdb])
X_test = pd.concat([X_test_rt, X_test_imdb])
y_train = pd.concat([y_train_rt, y_train_imdb])
y_test = pd.concat([y_test_rt, y_test_imdb])

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipeline = Pipeline([('vectorizer', vectorizer), 
                     ('classifier', LogisticRegression())])
model = pipeline.fit(X=X_train, y=y_train)

In [30]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.84      0.75      0.79     12576
          1       0.84      0.90      0.87     17946

avg / total       0.84      0.84      0.84     30522



In [31]:
y_pred = model.predict(X_test_rt)

print(classification_report(y_test_rt, y_pred))

             precision    recall  f1-score   support

          0       0.78      0.64      0.71      7521
          1       0.81      0.90      0.85     13001

avg / total       0.80      0.80      0.80     20522



In [32]:
y_pred = model.predict(X_test_imdb)

print(classification_report(y_test_imdb, y_pred))

             precision    recall  f1-score   support

          0       0.92      0.90      0.91      5055
          1       0.90      0.92      0.91      4945

avg / total       0.91      0.91      0.91     10000

